In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import pandas as pd

In [4]:
# Define the file paths
file1_path = "/content/drive/My Drive/data/train_data.csv"
file2_path = "/content/drive/My Drive/data/test_data.csv"
file3_path = "/content/drive/My Drive/data/train_labels.csv"

# # Load the data into Pandas DataFrames
# train_data = pd.read_csv(file1_path)
# test_data = pd.read_csv(file2_path)
# train_labels = pd.read_csv(file3_path)


In [5]:
train_data = pd.read_csv("/content/drive/My Drive/data/train_data.csv", header=None)
train_labels = pd.read_csv("/content/drive/My Drive/data/train_labels.csv", header=None)
test_data = pd.read_csv("/content/drive/My Drive/data/test_data.csv", header=None)

print('train_data.shape: {}'.format(train_data.shape))
print('train_labels.shape: {}'.format(train_labels.shape))
print('test_data.shape: {}'.format(test_data.shape))


train_data.shape: (160, 22036)
train_labels.shape: (160, 1)
test_data.shape: (40, 22036)


In [6]:
# imports:

from sklearn.multiclass import OneVsRestClassifier
from xgboost import XGBClassifier
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV


In [7]:
X = train_data
y = train_labels

print("X shape: ", X.shape)
print("y shape: ", y.shape)

X shape:  (160, 22036)
y shape:  (160, 1)


In [32]:
# Train/Test split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.30, random_state=42)

In [27]:
print("X_train shape: ", X_train.shape)
print("X_test shape: ", X_val.shape)
print("y_train shape: ", y_train.shape)
print("y_test shape: ", y_val.shape)

X_train shape:  (128, 22036)
X_test shape:  (32, 22036)
y_train shape:  (128, 1)
y_test shape:  (32, 1)


In [33]:
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression

pca = PCA(n_components=90)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_val)

logreg = LogisticRegression(max_iter=5000)

param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]
}
grid_search = GridSearchCV(logreg, param_grid, scoring='accuracy', cv=5)
grid_search.fit(X_train_pca, y_train[0].tolist())

print("Best Parameters: ", grid_search.best_params_)
print("Best Score: ", grid_search.best_score_)

# Train the final model with the best parameters
best_logreg = grid_search.best_estimator_
best_logreg.fit(X_train_pca, y_train[0].tolist())
test_acc = accuracy_score(y_val, best_logreg.predict(X_test_pca))
print(f"Test Accuracy: {test_acc:.3f}")

Best Parameters:  {'C': 100}
Best Score:  0.6948616600790514
Test Accuracy: 0.792


In [43]:
# Define PCA and XGBoost models already fine tuned:

xgb_model = XGBClassifier(objective='reg:logistic',
                          num_class=5,  # Number of classes in your multi-class classification problem
                          learning_rate=0.03,
                          max_depth=3,
                          min_child_weight=3,
                          n_estimators=150,
                          reg_alpha=0.01,
                          n_jobs=4
                          )

# Define the pipeline including PCA and XGBoost
pipeline_02 = Pipeline([
    ('pca', pca),
    ('logistic', logreg),
    ('xgb', xgb_model)
])



In [45]:
# Fit the classifier to your training data
pipeline_02.fit(X_train, y_train)

TypeError: All intermediate steps should be transformers and implement fit and transform or be the string 'passthrough' 'LogisticRegression(max_iter=5000)' (type <class 'sklearn.linear_model._logistic.LogisticRegression'>) doesn't

In [44]:
# Evaluate the best model on the validation set
val_predictions = pipeline_02.predict(X_val)
val_accuracy = accuracy_score(y_val, val_predictions)
print("Validation accuracy: {:.2f}".format(val_accuracy))


AttributeError: 'LogisticRegression' object has no attribute 'transform'

In [28]:
# Now we can use ovr_classifier.predict() to predict labels for the test data
predicted_labels = pipeline_02.predict(test_data)
predicted_labels.shape

(40,)

In [ ]:
test_accuracy = accuracy_score(test_data, predicted_labels)
print("Test accuracy:", test_accuracy)

ValueError: Classification metrics can't handle a mix of continuous-multioutput and multiclass targets